In [ ]:
import mysql.connector
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
load_dotenv()

# os.getenv()를 사용해 .env 파일에 저장된 비밀번호를 가져옵니다.
db_password = os.getenv('DB_PASSWORD')


mydb = mysql.connector.connect(
      host='localhost',
      user='root',
      password=db_password,  # .env 파일에서 불러온 비밀번호 변수 사용
      db = 'mysql')
      

In [27]:
import mysql.connector as mc

# 1) DB 없이 연결
mydb = mc.connect(
    host='127.0.0.1', port=3306,
    user='root',
    password=db_password,
    autocommit=True
)

with mydb.cursor() as cur:
    # 2) 데이터베이스 생성
    cur.execute("""
        CREATE DATABASE IF NOT EXISTS data
        DEFAULT CHARACTER SET utf8mb4
        COLLATE utf8mb4_unicode_ci
    """)
    # 3) 선택
    cur.execute("USE data")
    # 4) 테이블 생성
    cur.execute("""
        CREATE TABLE IF NOT EXISTS tb_student(
            id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(32),
            age INT,
            address VARCHAR(64)
        ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
    """)

# 5) (권장) 데이터베이스를 명시해 재연결
mydb1 = mc.connect(
    host='127.0.0.1', port=3306,
    user='root',
    password=db_password,
    database='data',
    autocommit=True
)


In [ ]:
sql = '''
CREATE TABLE tb_student(
    id int primary key auto_increment not null,
    name varchar(32),
    age int,
    address varchar(32)
) ENGINE = InnoDB DEFAULT CHARSET = utf8
'''
with mydb.cursor() as cursor:
    cursor.execute(sql)



DB_HOST, DB_USER, DB-NAME 이런거 확인해서 DatabaseConfig()만들려는 방법좀 알려줘

In [25]:
# mydb는 이미 연결된 상태라고 가정합니다.
# (autocommit=False면, 아래처럼 commit()을 반드시 호출해야 반영됩니다.)

# 1) 단건 삽입
insert_sql = "INSERT INTO tb_student (name, age, address) VALUES (%s, %s, %s)"
one = ("홍길동", 24, "서울시 중구")
with mydb.cursor() as cur:
    cur.execute(insert_sql, one)
    mydb.commit()  # autocommit=False인 경우 필수
    print("삽입된 id:", cur.lastrowid)

# 2) 다건 삽입
many = [
    ("김민수", 22, "서울"),
    ("이영희", 23, "부산"),
    ("박준호", 21, "대구"),
]
with mydb.cursor() as cur:
    cur.executemany(insert_sql, many)
    mydb.commit()
    print("삽입된 행 수:", cur.rowcount)

# 3) 삽입 확인 (파이썬에서 즉시 조회)
with mydb.cursor() as cur:
    cur.execute("SELECT id, name, age, address FROM tb_student ORDER BY id")
    for row in cur.fetchall():
        print(row)


삽입된 id: 1
삽입된 행 수: 3
(1, '홍길동', 24, '서울시 중구')
(2, '김민수', 22, '서울')
(3, '이영희', 23, '부산')
(4, '박준호', 21, '대구')


### 7.3 챗봇 학습툴 만들기
- 챗봇의 학습 데이터를 관리하는 툴
### 7.3.1 프로젝트 구조
- 이 절부터는 실습을 통해 챗봇 기능을 하나하나 만들어나간다.
- 소스 코드가 계속해서 이어지며, 경우에 따라서는 일부 소스만 보여줄 수 있다.
### 7.3.2 학습용 데이터베이스 설계 및 데이터 테이블 생성
- 이 절에서는 챗봇 엔진 답변 처리 과정에서 필요한 DB 구조를 설계
- 학습할 수 있는 툴을 만든다.


In [28]:
%pip install config

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pymysql
import openpyxl

from config.DatabaseConfig import *
def all_clear_train_data(db):
    # 기존 학습 데이터 삭제
    sql = '''
        delete from chatbot_train_data
'''
with db.cursor() as cursor:
    cursor.execute(sql)



ModuleNotFoundError: No module named 'config.DatabaseConfig'

In [ ]:
from konlpy.tag import Komoran
class Preprocess:
    def __init__(self, userdic = None):
        self.komoran (userdic = userdic)

        # 제외할 품사
        #  참조:
        # 관계언 제거, 기호 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS','JKC','JKG','JKO'
        ]

### 8.8.1 통신 프로토콜 정의
- 우선 챗봇 엔진 서버와 통신하기 위해 프로토콜부터 정의
- 프로토콜이란 서버와 클라이언트 간에 통신을 하기 위해 필요한 규약
### 8.8.2 다중 접속을 위한 TCP 소켓 서버
- 싱글 스레드 방식
- 싱글 스레드로는 다수의 챗봇 클라이언트 서비스 요청을 한번에 처리할 수 없다. 